In [1]:
import os, sys, json
from sys import platform
from dotenv import dotenv_values
from transformers import AutoModelForCausalLM, AutoTokenizer
_base_path = '\\'.join(os.getcwd().split('\\')[:-1]) + '\\' if platform == 'win32' else '/'.join(os.getcwd().split('/')[:-1]) + '/'
sys.path.append(_base_path)
from src.helpers.utils import get_config, get_device

In [3]:
_config = get_config()
_env = dotenv_values(os.path.join(_base_path, '.env'))
json_path = os.path.join(_base_path, _config.get('JSON_FOLDER'))
if platform == 'win32':
	json_path = json_path.replace('/', '\\')

In [4]:
PROMPT = "Using the provided JSON data from a brain MRI study, please generate a comprehensive report detailing the segmentation results of a brain tumor. Describe the tumor’s presence in various brain regions according to the Julich-Brain Atlas, noting the percentages of the tumor within each region and the impact on the region. While discussing the confidence of the segmentation and the model used, keep the explanations high-level and avoid overly technical details. The goal is to enhance the explainability of the AI segmentation to support clinical decision-making in a manner that can be easily understood by medical professionals. "

In [5]:
with open(os.path.join(json_path, 'SegResNet_sample_2.json'), 'r') as f:
	d = json.load(f)
	PROMPT += str(d)

In [6]:
device = get_device()
model_id = 'BioMistral/BioMistral-7B'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

In [11]:
inputs = tokenizer.apply_chat_template(
	[{'role': 'user', 'content': PROMPT}],
	tokenize = True,
	add_generation_prompt = True,
	return_tensors = 'pt'
).to(device)
outputs = model.generate(
	inputs,
	max_new_tokens = 1024,
	pad_token_id = tokenizer.eos_token_id
)
readable_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(readable_output.split('[/INST] ')[1].replace('. ', '. \n'))

The AI-based segmentation model was able to accurately identify the tumor in the provided brain MRI scan. 
The tumor was found to be present in five different brain regions according to the Julich-Brain Atlas, including Temporal-to-Parietal (GapMap) right, Area hOc4la (LOC) right, Area hOc4lp (LOC) right, Area hIP4 (IPS) right, and Area PGp (IPL) right. 
The percentage of the tumor within each region was 49.22%, 14.64%, 12.62%, 6.36%, and 4.98%, respectively. 
The impact of the tumor on the brain regions was also assessed, with the percentage of the region affected by the tumor being 15.55%, 89.44%, 84.2%, 97.44%, and 22.97%, respectively.


In [10]:
print(PROMPT)

Using the provided JSON data from a brain MRI study, please generate a comprehensive report detailing the segmentation results of a brain tumor. Describe the tumor’s presence in various brain regions according to the Julich-Brain Atlas, noting the percentages of the tumor within each region and the impact on the region. While discussing the confidence of the segmentation and the model used, keep the explanations high-level and avoid overly technical details. The goal is to enhance the explainability of the AI segmentation to support clinical decision-making in a manner that can be easily understood by medical professionals. {'MRI_Scan': {'Tumor_Details': {'Spatial_Distribution': [{'Region': 'Temporal-to-Parietal (GapMap) right', 'Percentage_of_Tumor': 49.22, 'Percentage_of_Region_Affected': 15.55}, {'Region': 'Area hOc4la (LOC) right', 'Percentage_of_Tumor': 14.64, 'Percentage_of_Region_Affected': 89.44}, {'Region': 'Area hOc4lp (LOC) right', 'Percentage_of_Tumor': 12.62, 'Percentage_o